Code is based on [The following example](http://sujitpal.blogspot.nl/2016/04/predicting-movie-tags-from-plots-using.html)

In [84]:
import gensim
import json
from random import shuffle
import numpy as np

In [140]:
def construct_topic_knowledge(fname):
    file = '../data_resources/topics/{}.json'.format(fname)
    data = json.load(open(file))
    
    topics = {}
    
    for obj in data:
        main = obj['main_topic']
        sub = obj['sub_topic']
        
        if not main in topics:
            topics[main] = []
        
        if not sub in topics[main]:
            topics[main].append(sub)
    
    return topics
        

def read_corpus(fname, tokens_only=False, topic_type='sub_topic'):
    file = '../data_resources/topics/{}.json'.format(fname)
    data = json.load(open(file))
    
    for obj in data:
        if tokens_only:
            yield gensim.utils.simple_preprocess(obj['content'])
        else:
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(obj['content']), [obj[topic_type]])
            
    
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
    
def tokenize_tags(label):
    tags = label.split("::")
    tags = map(lambda tok: mark_tag(tok), tags)
    return tags

def jaccard_similarity(labels, preds):
    lset = set(labels)
    pset = set(preds)
    return len(lset.intersection(pset)) / len(lset.union(pset))


In [141]:
topics = construct_topic_knowledge('vng_training_cleaned_no_news')

In [142]:
corpus_sub = list(read_corpus('vng_training_cleaned_no_news'))
corpus_main = list(read_corpus('vng_training_cleaned_no_news', topic_type='main_topic'))

In [143]:
split = int(0.9*len(corpus))

train_corpus_sub = corpus_sub[0:split]
train_corpus_main = corpus_main[0:split]

test_corpus_sub = corpus_sub[split:]
test_corpus_main = corpus_main[split:]

In [144]:
model_sub = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=100)
model_sub.build_vocab(train_corpus_sub)
%time model_sub.train(train_corpus_sub, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 2min 8s, sys: 1.74 s, total: 2min 9s
Wall time: 49.9 s


42023252

In [145]:
model_main = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=100)
model_main.build_vocab(train_corpus_main)
%time  model_main.train(train_corpus_main, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 2min 10s, sys: 1.87 s, total: 2min 12s
Wall time: 51.5 s


42021746

In [160]:
def top_tags(model_main, model_sub, doc, topics_knowledge, top_n=1):
    
    pred_vec_main = model_main.infer_vector(doc.words)
    pred_tags_main = model_main.docvecs.most_similar([pred_vec_main], topn=5)

    pred_vec_sub = model_sub.infer_vector(doc.words)
    pred_tags_sub = model_sub.docvecs.most_similar([pred_vec_sub], topn=10)

    final_sub_tags = []
    
    for pred_tag_sub in pred_tags_sub:
        additional_certainty = 0

        for pred_tag_main in pred_tags_main:
            
            if pred_tag_sub[0] in topics_knowledge[pred_tag_main[0]]:
                
                score = pred_tag_sub[1] + pred_tag_main[1]
                
                final_sub_tags.append((pred_tag_sub[0], score))
                

    while len(final_sub_tags) > top_n:
        min_certainty = 999
        min_certainty_tag = None

        for tag in final_sub_tags:
            if tag[1] < min_certainty:
                min_certainty = tag[1]
                min_certainty_tag = tag

        final_sub_tags.remove(tag)

    return final_sub_tags

In [163]:
def evaluate_dual_model(model_main, model_sub, corpus_test, topics_knowledge, top_n = 1):
    tot_sim = 0
    for doc in corpus_test:
        pred_tags = top_tags(model_main, model_sub, doc, topics_knowledge, top_n=1)
        
        print(pred_tags, doc.tags)
        
        sim = jaccard_similarity(doc.tags, [x[0] for x in pred_tags])
        tot_sim += sim
    print("Average Similarity on Test Set: {}".format(tot_sim / len(corpus)))

In [164]:
evaluate_dual_model(model_main, model_sub, test_corpus_sub, topics, top_n = 1)

[('gemeentelijke-samenwerking', 1.0751418471336365)] ['energie-en-klimaat']
[('rijksbelastingen', 1.0956618785858154)] ['jeugdhulp']
[('gemeentewet', 1.1254003643989563)] ['handhaving-en-fraudewet']
[('beschermd-wonen-maatschappelijke-opvang', 1.22946435213089)] ['beschermd-wonen-maatschappelijke-opvang-ggz']
[('verkiezingen-referendum', 1.2722867131233215)] ['verkiezingen-referendum']
[('regionale-economie-stimuleren', 1.333158552646637)] ['regionale-economie-stimuleren']
[('gemeentefinancien-in-beleidsdossiers', 1.151347279548645)] ['jeugdhulp']
[('rijksbelastingen', 1.2700991034507751)] ['begroting-en-verantwoording']
[('europese-kennisnetwerken', 1.147822916507721)] ['europese-kennisnetwerken']
[('wet-op-de-lijkbezorging', 1.1653724312782288)] ['asielbeleid-en-integratie']
[('nieuws', 0.3466237857937813)] ['wmo-2015']
[('handhaving-arbeidsverplichting', 1.3132140040397644)] ['arbeidsmarktbeleid']
[('jeugdhulp', 1.2803852558135986)] ['jeugdhulp']
[('jeugdhulp', 1.2507495284080505)] 

[('individueel-keuzebudget-ikb', 1.0367276668548584)] ['individueel-keuzebudget-ikb']
[('informatieveiligheid-privacy', 1.3155252933502197)] ['informatieveiligheid']
[('nieuws', 0.3466237857937813)] ['wmo-2015']
[('rijksbelastingen', 1.3731866478919983)] ['rijksbelastingen']
[('asielbeleid-en-integratie-test', 1.1448787450790405)] ['asielbeleid-en-integratie']
[('kabels-en-leidingen', 0.8972015380859375)] ['dienstverlening-aan-inwoners-en-ondernemers']
[] ['handhaving-en-fraudewet']
[('gemeentefinancien-in-beleidsdossiers', 1.1754682064056396)] ['isd-informatievoorziening-sociaal-domein']
[('leerlingenvervoer', 1.1078158020973206)] ['leerlingenvervoer']
[('overheidsaansprakelijkheid', 1.1470937132835388)] ['werk-voor-arbeidsbeperkten']
[('handhaving-arbeidsverplichting', 1.0409948825836182)] ['asielbeleid-en-integratie']
[('gezondheidsbeleid', 1.1773143410682678)] ['asielbeleid-en-integratie']
[('ro-wet-en-regelgeving', 1.2976723909378052)] ['omgevingswet']
[('europese-stedelijke-agend

[('overheidsaansprakelijkheid', 1.2499867677688599)] ['nieuws']
[('jeugdhulp', 1.3429869413375854)] ['jeugdhulp']
[('kindcentra', 1.2455862760543823)] ['onderwijsachterstandvve']
[('privacy-avg', 1.2070306539535522)] ['basisregistraties']
[('generieke-digitale-infrastructuur-gdi', 1.3182064294815063)] ['dienstverlening-aan-inwoners-en-ondernemers']
[('handhaving-arbeidsverplichting', 1.1575633883476257)] ['armoedebeleid-en-schuldhulpverlening']
[('fiscaal-procesrecht', 1.2226185202598572)] ['gemeentewet']
[('rijksbelastingen', 1.206170380115509)] ['gemeentefonds-en-btw-compensatiefonds']
[('landbouw-en-veehouderij', 1.1347110867500305)] ['ondersteuningsprogramma-energie']
[('sociale-zekerheid', 1.1699901819229126)] ['werk-voor-arbeidsbeperkten']
[('architectuur-archeologie-en-monumenten', 1.0717507004737854)] ['onderwijshuisvesting']
[('inkoop-kwaliteit-en-toezicht-wmo', 1.0959367156028748)] ['inkoop-kwaliteit-en-toezicht']
[] ['nieuws']
[('overheidsaansprakelijkheid', 1.16035288572311

In [174]:
# SIMPLE DOCTAG2VEC MODEL WORKS BETTER THAN COMBINING MAIN TOPICS WITH SUB TOPICS.

def evaluate_model(model, corpus):
    tot_sim = 0
    for doc in corpus:
        pred_vec = model.infer_vector(doc.words)
        pred_tags = model.docvecs.most_similar([pred_vec], topn=1)
        sim = jaccard_similarity(doc.tags, [x[0] for x in pred_tags])
        tot_sim += sim
    print("Average Similarity on Test Set: {}".format(tot_sim / len(corpus)))
    
evaluate_model(model_sub, test_corpus_sub)

Average Similarity on Test Set: 0.21172638436482086


In [34]:
# print out random test result
for i in range(10):
    docid = np.random.randint(len(train_corpus))
    pred_vec = model.infer_vector(train_corpus[docid].words)
#     actual_tags = map(lambda x: unmark_tag(x), train_corpus[docid].tags)
    actual_tags = train_corpus[docid].tags
    pred_tags = model.docvecs.most_similar([pred_vec], topn=5)
    
    print("Actual tags: {}".format(", ".join(actual_tags)))
    print("Predicted tags: {}".format(pred_tags))
    print('--')

Actual tags: huisvestingswet
Predicted tags: [('huisvestingswet', 0.8789634704589844), ('overheidsaansprakelijkheid', 0.8708899021148682), ('landbouw-en-veehouderij', 0.8487333059310913), ('luchtkwaliteit', 0.8310826420783997), ('grondbeleidgrondzaken', 0.8105636835098267)]
--
Actual tags: isd-informatievoorziening-sociaal-domein
Predicted tags: [('overheidsaansprakelijkheid', 0.7986610531806946), ('landbouw-en-veehouderij', 0.7889240384101868), ('winkeltijdenwet', 0.7824044823646545), ('wet-openbaarheid-van-bestuur-wob', 0.7823542952537537), ('awb', 0.7720432877540588)]
--
Actual tags: energie-en-klimaat
Predicted tags: [('bevolkingsdaling', 0.7620388865470886), ('leegstand-en-herbestemming', 0.7427639365196228), ('leegstand-transformatie-en-herbestemming', 0.7198410630226135), ('luchtkwaliteit', 0.7092674970626831), ('gemeenten-kapitaalmarkt', 0.7028281688690186)]
--
Actual tags: asielbeleid-en-integratie
Predicted tags: [('overheidsaansprakelijkheid', 0.8651185035705566), ('kernbele